<a href="https://colab.research.google.com/github/khareyash05/Multi-Modal-Parkinson-Disease-Detection-using-Transformers/blob/main/image_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [83]:
# ! mkdir ~/.kaggle

In [84]:
# ! cp kaggle.json ~/.kaggle/

In [85]:
# ! chmod 600 ~/.kaggle/kaggle.json

In [86]:
# ! kaggle datasets download -d kmader/parkinsons-drawings

In [87]:
# !unzip parkinsons-drawings.zip

In [88]:
train_dir_spiral = '/content/spiral/training'
test_dir_spiral = '/content/spiral/testing'

In [89]:
import os
Name=[]
for file in os.listdir(train_dir_spiral):
    Name+=[file]
print(Name)
print(len(Name))

['parkinson', 'healthy']
2


In [90]:
N=[]
for i in range(len(Name)):
    N+=[i]
    
normal_mapping=dict(zip(Name,N)) 
reverse_mapping=dict(zip(N,Name)) 

def mapper(value):
    return reverse_mapping[value]

In [91]:
import os
import numpy as np
import pandas as pd
import cv2
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
# from tf.keras.utils import to_categorical
# from tf.keras.preprocessing.image import load_img, img_to_array
# from tf.python.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.model_selection import train_test_split

In [92]:
dataset_sp=[]
count=0
for file in os.listdir(train_dir_spiral):
    path=os.path.join(train_dir_spiral,file)
    for im in os.listdir(path):
        image=tf.keras.preprocessing.image.load_img(os.path.join(path,im), grayscale=False, color_mode='rgb', target_size=(224,224))
        image=tf.keras.preprocessing.image.img_to_array(image)
        image=image/255.0
        dataset_sp.append([image,count])
    count=count+1
    
testset_sp=[]
count=0
for file in os.listdir(test_dir_spiral):
    path=os.path.join(test_dir_spiral,file)
    for im in os.listdir(path):
        image=tf.keras.preprocessing.image.load_img(os.path.join(path,im), grayscale=False, color_mode='rgb', target_size=(224,224))
        image=tf.keras.preprocessing.image.img_to_array(image)
        image=image/255.0
        testset_sp.append([image,count])
    count=count+1

In [93]:
data_sp,labels_sp0=zip(*dataset_sp)
test_sp,tlabels_sp0=zip(*testset_sp)

In [94]:
labels_sp1=tf.keras.utils.to_categorical(labels_sp0)
data_sp=np.array(data_sp)
labels_sp=np.array(labels_sp1)

tlabels_sp1=tf.keras.utils.to_categorical(tlabels_sp0)
test_sp=np.array(test_sp)
tlabels_sp=np.array(tlabels_sp1)

In [95]:
trainx_sp,testx_sp,trainy_sp,testy_sp=train_test_split(data_sp,labels_sp,test_size=0.2,random_state=44)

In [96]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True,vertical_flip=True,rotation_range=20,zoom_range=0.2,
                        width_shift_range=0.2,height_shift_range=0.2,shear_range=0.1,fill_mode="nearest")

In [97]:
pretrained_model3 = tf.keras.applications.mobilenet_v2.MobileNetV2(input_shape=(224,224,3),include_top=False,weights='imagenet',pooling='avg')
pretrained_model3.trainable = False

In [98]:
inputs3 = pretrained_model3.input
x3 = tf.keras.layers.Dense(128, activation='relu')(pretrained_model3.output)
outputs3 = tf.keras.layers.Dense(2, activation='sigmoid')(x3)
model3 = tf.keras.Model(inputs=inputs3, outputs=outputs3)

In [99]:
model3.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_5[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                           

In [100]:
model3.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [101]:
his3=model3.fit(datagen.flow(trainx_sp,trainy_sp,batch_size=32),validation_data=(testx_sp,testy_sp),epochs=30)

Epoch 1/30
2/2 [==============================] - 8s 2s/step - loss: 0.9217 - accuracy: 0.5088 - val_loss: 0.4897 - val_accuracy: 0.8000
Epoch 2/30
2/2 [==============================] - 3s 2s/step - loss: 0.6217 - accuracy: 0.5965 - val_loss: 0.6625 - val_accuracy: 0.5333
Epoch 3/30
2/2 [==============================] - 3s 2s/step - loss: 0.6298 - accuracy: 0.6316 - val_loss: 0.3601 - val_accuracy: 0.8667
Epoch 4/30
2/2 [==============================] - 3s 1s/step - loss: 0.4465 - accuracy: 0.7895 - val_loss: 0.3190 - val_accuracy: 0.9333
Epoch 5/30
2/2 [==============================] - 4s 1s/step - loss: 0.4988 - accuracy: 0.7895 - val_loss: 0.3497 - val_accuracy: 0.8000
Epoch 6/30
2/2 [==============================] - 3s 2s/step - loss: 0.4209 - accuracy: 0.8070 - val_loss: 0.4722 - val_accuracy: 0.7333
Epoch 7/30
2/2 [==============================] - 3s 2s/step - loss: 0.4180 - accuracy: 0.7895 - val_loss: 0.3700 - val_accuracy: 0.8000
Epoch 8/30
2/2 [=========================

In [102]:
import pickle 
pickle.dump(model3, open('img_model.pkl', 'wb'))

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......add
.........vars
......add_1
.........vars
......add_2
.........vars
......add_3
.........vars
......add_4
.........vars
......add_5
.........vars
......add_6
.........vars
......add_7
.........vars
......add_8
.........vars
......add_9
.........vars
......batch_normalization
.........vars
............0
............1
............2
............3
......batch_normalization_1
.........vars
............0
............1
............2
............3
......batch_normalization_10
.........vars
............0
............1
............2
............3
......batch_normalization_11
.........vars
............0
............1
............2
............3
......batch_normalization_12
.........vars
............0
............1
............2
............3
......batch_normalization_13
.........vars
............0
............1
............2
............3
......batch_normalization_14
.........vars
............0
............1
.....